#from allcools page: https://lhqing.github.io/ALLCools/cell_level/step_by_step/100kb/03-HighlyVariableFeatureSelection.html
Calculate Highly Variable Features And Get mC Fraction AnnData¶

Purpose
The purpose of this step is to select highly variable features (HVF) and generate cell-by-feature methylation fraction matrix for clustering. The highly variable features are selected by comparing feature’s normalized dispersion among cells.

In [ ]:
Input¶
Filtered cell metadata;

MCDS files;

Feature list from basic feature filtering

Output
cell-by-HVF methylation fraction matrix stored in AnnData format, e.g., mCH adata and mCG adata.

In [5]:
cd /share/lasallelab/Ensi/project/allcools/oocytes/

/share/lasallelab/Ensi/project/allcools/oocytes


In [26]:
import pathlib
import pandas as pd
import dask
from ALLCools.mcds import MCDS

In [9]:
# If True, will load all data into memory.
# Computation will be much faster, but also very memory intensive, only use this for small number of cells (<10,000)
load = True

# change this to the path to your filtered metadata
metadata_path = 'CellMetadata.PassQC_oocyte.csv'

# change this to the paths to your MCDS files
mcds_path = 'mcds'

# Feature list after basic filter
feature_path = 'FeatureList.BasicFilter.txt'

# Dimension name used to do clustering
obs_dim = 'cell'  # observation
var_dim = 'chrom100k'  # feature

# HVF method:
# SVR: regression based
# Bins: normalize dispersion per bin
hvf_method = 'SVR'
mch_pattern = 'CHN'
mcg_pattern = 'CGN'
n_top_feature = 25000

# Downsample cells
downsample = 20000

In [10]:
metadata = pd.read_csv(metadata_path, index_col=0)
total_cells = metadata.shape[0]
print(f'Metadata of {total_cells} cells')

Metadata of 120 cells


In [12]:
metadata.head()


,Date,Type of sample,WellID,WellBarcode,AnimalID,Type,TotalRead,mCGFrac,Group,SampleName
SampleID,,,,,,,,,,
3889744700PO1NC5D1S,44700,Pooled Oocytes (5) #1,D1,CAGTCACA,38897,PO,10700000,0.693,Stressed,3889744700PO1NC5D1S_1_val_1_bismark_bt2_pe.all...
3905345005PO1NC2I1S,45005,Pooled Oocytes (2) #1,I1,TGATAGGC,39053,PO,2700000,0.623,Stressed,3905345005PO1NC2I1S_1_val_1_bismark_bt2_pe.all...
4129944963PO1NC3N1S,44963,Pooled Oocytes (3) #1,N1,ATTCCGCT,41299,PO,2900000,0.626,Stressed,4129944963PO1NC3N1S_1_val_1_bismark_bt2_pe.all...
4129944963PO2NC4M1S,44963,Pooled Oocytes (4) #2,M1,CACGCAAT,41299,PO,2300000,0.684,Stressed,4129944963PO2NC4M1S_1_val_1_bismark_bt2_pe.all...
4129944963PO3NC3O1S,44963,Pooled Oocytes (3) #3,O1,AGAAGGAC,41299,PO,4800000,0.641,Stressed,4129944963PO3NC3O1S_1_val_1_bismark_bt2_pe.all...


In [15]:
use_features = pd.read_csv(feature_path, header=None, index_col=0).index
use_features.name = var_dim
use_features

Index(['chr1_0', 'chr1_1', 'chr1_2', 'chr1_3', 'chr1_4', 'chr1_5', 'chr1_6',
       'chr1_7', 'chr1_8', 'chr1_9',
       ...
       'chrX_1524', 'chrX_1525', 'chrX_1526', 'chrX_1527', 'chrX_1528',
       'chrX_1529', 'chrX_1530', 'chrX_1531', 'chrX_1532',
       'chrX_NW_021160383v1_random_0'],
      dtype='object', name='chrom100k', length=28068)

In [20]:
#MCDS
total_mcds = MCDS.open(mcds_path,
                       var_dim=var_dim,
                       use_obs=metadata.index).sel({var_dim: use_features})
total_mcds

<xarray.MCDS> Size: 88MB
Dimensions:          (cell: 120, chrom100k: 28068, count_type: 2, mc_type: 3)
Coordinates:
  * cell             (cell) <U20 10kB '4636344664PO1NC5L9C' ... '4211144957SO...
  * chrom100k        (chrom100k) <U29 3MB 'chr1_0' ... 'chrX_NW_021160383v1_r...
    chrom100k_chrom  (chrom100k) <U27 3MB ...
    chrom100k_end    (chrom100k) int64 225kB ...
    chrom100k_start  (chrom100k) int64 225kB ...
  * count_type       (count_type) <U3 24B 'mc' 'cov'
  * mc_type          (mc_type) <U3 36B 'CGN' 'CAN' 'CHN'
Data variables:
    chrom100k_da     (cell, chrom100k, mc_type, count_type) uint32 81MB dask.array<chunksize=(31, 7758, 1, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [16]:
#add mC Rate 
total_mcds.add_mc_rate(var_dim=var_dim,
                       normalize_per_cell=True,
                       clip_norm_value=10)

total_mcds

<xarray.MCDS> Size: 168MB
Dimensions:            (cell: 120, chrom100k: 28068, count_type: 2, mc_type: 3)
Coordinates:
  * cell               (cell) <U20 10kB '4636344664PO1NC5L9C' ... '4211144957...
  * chrom100k          (chrom100k) <U29 3MB 'chr1_0' ... 'chrX_NW_021160383v1...
    chrom100k_chrom    (chrom100k) <U27 3MB ...
    chrom100k_end      (chrom100k) int64 225kB ...
    chrom100k_start    (chrom100k) int64 225kB ...
  * count_type         (count_type) <U3 24B 'mc' 'cov'
  * mc_type            (mc_type) <U3 36B 'CGN' 'CAN' 'CHN'
Data variables:
    chrom100k_da       (cell, chrom100k, mc_type, count_type) uint32 81MB dask.array<chunksize=(31, 7758, 1, 1), meta=np.ndarray>
    chrom100k_da_frac  (cell, chrom100k, mc_type) float64 81MB dask.array<chunksize=(31, 7758, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [19]:
# we are not downsampleing, therefore: 
mcds = total_mcds
mcds

<xarray.MCDS> Size: 168MB
Dimensions:            (cell: 120, chrom100k: 28068, count_type: 2, mc_type: 3)
Coordinates:
  * cell               (cell) <U20 10kB '4636344664PO1NC5L9C' ... '4211144957...
  * chrom100k          (chrom100k) <U29 3MB 'chr1_0' ... 'chrX_NW_021160383v1...
    chrom100k_chrom    (chrom100k) <U27 3MB ...
    chrom100k_end      (chrom100k) int64 225kB ...
    chrom100k_start    (chrom100k) int64 225kB ...
  * count_type         (count_type) <U3 24B 'mc' 'cov'
  * mc_type            (mc_type) <U3 36B 'CGN' 'CAN' 'CHN'
Data variables:
    chrom100k_da       (cell, chrom100k, mc_type, count_type) uint32 81MB dask.array<chunksize=(31, 7758, 1, 1), meta=np.ndarray>
    chrom100k_da_frac  (cell, chrom100k, mc_type) float64 81MB dask.array<chunksize=(31, 7758, 1), meta=np.ndarray>
Attributes:
    obs_dim:  cell
    var_dim:  chrom100k

In [21]:
# Highly variable features
#mCH

if hvf_method == 'SVR':
    # use SVR based method
    mch_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                     mc_type=mch_pattern,
                                     n_top_feature=n_top_feature,
                                     plot=True)
else:
    # use bin based method
    mch_hvf = mcds.calculate_hvf(var_dim=var_dim,
                                 mc_type=mch_pattern,
                                 min_mean=0,
                                 max_mean=5,
                                 n_top_feature=n_top_feature,
                                 bin_min_features=5,
                                 mean_binsize=0.05,
                                 cov_binsize=100)

/share/lasallelab/Ensi/anaconda3/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))
/share/lasallelab/Ensi/anaconda3/lib/python3.11/site-packages/dask/core.py:127: RuntimeWarning: invalid value encountered in divide
  return func(*(_execute_task(a, cache) for a in args))


Fitting SVR with gamma 0.0356, predicting feature dispersion using mc_frac_mean and cov_mean.
Total Feature Number:     28068
Highly Variable Feature:  25000 (89.1%)


In [24]:
#save AnnData

total_mcds.coords[f'{var_dim}_{mch_pattern}_feature_select'] = mcds.coords[
    f'{var_dim}_{mch_pattern}_feature_select']

In [23]:
mch_adata = total_mcds.get_adata(mc_type=mch_pattern,
                           var_dim=var_dim,
                           select_hvf=True)

mch_adata.write_h5ad(f'mCH.HVF.h5ad')

mch_adata


KeyError: 'chrom100k_frac or chrom100k_da_frac not found in MCDS.'

In [ ]:
# mCG
if hvf_method == 'SVR':
    # use SVR based method
    mcg_hvf = mcds.calculate_hvf_svr(var_dim=var_dim,
                                     mc_type=mcg_pattern,
                                     n_top_feature=n_top_feature,
                                     plot=True)
else:
    # use bin based method
    mcg_hvf = mcds.calculate_hvf(var_dim=var_dim,
                                 mc_type=mcg_pattern,
                                 min_mean=0,
                                 max_mean=5,
                                 n_top_feature=n_top_feature,
                                 bin_min_features=5,
                                 mean_binsize=0.02,
                                 cov_binsize=20)

In [ ]:
# save AnnData
total_mcds.coords[f'{var_dim}_{mcg_pattern}_feature_select'] = mcds.coords[
    f'{var_dim}_{mcg_pattern}_feature_select']

In [ ]:
mcg_adata = total_mcds.get_adata(mc_type=mcg_pattern,
                                 var_dim=var_dim,
                                 select_hvf=True)

mcg_adata.write_h5ad(f'mCG.HVF.h5ad')

mcg_adata

In [27]:
metadata

,Date,Type of sample,WellID,WellBarcode,AnimalID,Type,TotalRead,mCGFrac,Group,SampleName
SampleID,,,,,,,,,,
3889744700PO1NC5D1S,44700,Pooled Oocytes (5) #1,D1,CAGTCACA,38897,PO,10700000,0.693,Stressed,3889744700PO1NC5D1S_1_val_1_bismark_bt2_pe.all...
3905345005PO1NC2I1S,45005,Pooled Oocytes (2) #1,I1,TGATAGGC,39053,PO,2700000,0.623,Stressed,3905345005PO1NC2I1S_1_val_1_bismark_bt2_pe.all...
4129944963PO1NC3N1S,44963,Pooled Oocytes (3) #1,N1,ATTCCGCT,41299,PO,2900000,0.626,Stressed,4129944963PO1NC3N1S_1_val_1_bismark_bt2_pe.all...
4129944963PO2NC4M1S,44963,Pooled Oocytes (4) #2,M1,CACGCAAT,41299,PO,2300000,0.684,Stressed,4129944963PO2NC4M1S_1_val_1_bismark_bt2_pe.all...
4129944963PO3NC3O1S,44963,Pooled Oocytes (3) #3,O1,AGAAGGAC,41299,PO,4800000,0.641,Stressed,4129944963PO3NC3O1S_1_val_1_bismark_bt2_pe.all...
...,...,...,...,...,...,...,...,...,...,...
4636344664SO3NC1K9C,44664,Single Oocyte #3,K9,TCAGACAC,46363,SO,4900000,0.689,Control,4636344664SO3NC1K9C_1_val_1_bismark_bt2_pe.all...
4660644938SO1NC1M9C,44938,Single Oocyte #1,M9,ATGCGTCA,46606,SO,43500000,0.691,Control,4660644938SO1NC1M9C_1_val_1_bismark_bt2_pe.all...
4660644938SO2NC1N9C,44938,Single Oocyte #2,N9,CACAGGAA,46606,SO,4800000,0.685,Control,4660644938SO2NC1N9C_1_val_1_bismark_bt2_pe.all...
